In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [4]:
import pickle
import codecs
import collections
import keras.preprocessing
from sklearn.model_selection import KFold
import csv

In [5]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras_contrib.layers import CRF
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,TimeDistributed,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dropout
from keras.models import load_model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding,TimeDistributed,Flatten
from keras.layers import BatchNormalization,Dropout


In [32]:
input_pbs = pd.read_csv('Thaipbs-tokenize_include_stop.csv')

In [33]:
input_pbs.drop('Unnamed: 0', axis=1, inplace=True)

In [34]:
input_pbs.head()

,Unnamed: 0.1,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar
0,0,ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควั...,05/04/2562,FRI,19:37,177,สิ่งแวดล้อม,"ฝุ่นเชียงใหม่,ฝุ่นPM2.5,ฝุ่นคลุมเมือง,เชียงใหม...",19,10,"ฝุ่น,PM,ทส.,เตรียม,ตั้ง,ศูนย์,แก้,ปัญหา,หมอก,ค...",12,47
1,1,"เลือกตั้ง 2562 : ""เพื่อไทย"" เตรียมยื่น กกต.จัด...",05/04/2562,FRI,19:20,702,การเมือง,"เลือกตั้ง62,เพื่อ่ไทย,กกต.,นับคะแนน,ไทยพีบีเอส...",19,6,"เลือกตั้ง,2562,เพื่อ,ไทย,เตรียม,ยื่น,กกต.,จัด,...",12,58
2,2,เตรียมเอาผิด รพ.เอกชน 58 แห่งไม่ส่งข้อมูลราคายา,05/04/2562,FRI,19:18,583,สาธารณสุข,"ยา,เวชภัณฑ์ฯ,โรงพยาบาล,ค่าบริการทางการแพทย์,กร...",19,9,"เตรียม,เอา,ผิด,รพ.เอกชน,แห่ง,ไม่,ส่ง,ข้อมูล,รา...",10,42
3,3,ตั้งข้อสังเกตปลดล็อกกัญชา เอื้อนายทุนหรือไม่?,05/04/2562,FRI,19:16,928,อาชญากรรม,"กัญชา,มูลนิธิข้าวขวัญ,ThaiPBSnews",19,3,"ตั้ง,ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,นาย,ทุน,ห...",11,43
4,4,"คำพิพากษาเต็ม ""เปรมชัย"" หลุดคดีครอบครองซากเสือดำ",05/04/2562,FRI,19:29,5163,สิ่งแวดล้อม,"เสือดำ,เปรมชัย,เขตรักษาพันธุ์สัตว์ป่าทุ่งใหญ่น...",19,5,"คำ,พิพากษา,เต็ม,เปรมชัย,หลุดคดี,ครอบครอง,ซาก,เ...",8,44


In [38]:
IQRval = input_pbs['view'].describe().loc['75%']-input_pbs['view'].describe().loc['25%']

In [39]:
outlierMin = max(input_pbs['view'].describe().loc['25%']-1.5*IQRval,0)
outlierMax = input_pbs['view'].describe().loc['75%']+1.5*IQRval
print(outlierMin,outlierMax)

0 3499.5


In [40]:
input_pbs = input_pbs[(input_pbs['view']<=outlierMax) & (input_pbs['view']>=outlierMin)]

In [41]:
input_pbs['numToken'].describe()

count    24511.000000
mean        11.389662
std          3.354892
min          1.000000
25%          9.000000
50%         11.000000
75%         13.000000
max         28.000000
Name: numToken, dtype: float64

In [42]:
input_pbs['view'].describe()

count    24511.000000
mean       755.928603
std        754.956352
min         20.000000
25%        228.000000
50%        454.000000
75%       1006.000000
max       3496.000000
Name: view, dtype: float64

In [43]:
MAXVIEW = input_pbs['view'].max()
MAXVIEW

3496

## Prepare data for Keras

In [44]:
inputText = []
for sent in input_pbs['token']:
    inputText.append(sent.split(','))

In [45]:
inputLabel = []
for view in input_pbs['view']:
    inputLabel.append(view/MAXVIEW)

In [46]:
print(len(inputText),len(inputLabel))

24511 24511


In [47]:
words = []
for sentence in inputText:
    for word in sentence:
        words.append(word)
        
word_count = list()
word_count.extend(collections.Counter(words).most_common(len(set(words))))
word_count = word_count[:len(word_count)//4]
word_count.append(("UNK",0))

train_word = set()
for i in word_count:
    train_word.add(i[0])

In [48]:
len(word_count)

4955

In [49]:
all_token = set()
for word in train_word:
    all_token.add(word)

In [50]:
all_token = list(all_token)
all_token.insert(0,'for padding')
all_token.insert(1,'UNK')

In [51]:
len(all_token)

4957

In [52]:
token_map = dict(zip(all_token, range(len(all_token))))
token_map_reverse = dict(zip(range(len(all_token)),all_token))

In [53]:
input_data = np.asarray(input_pbs['token'].str.split(','))

In [54]:
def count_word(sent):
    global all_token, token_map
    result = np.zeros(len(all_token))
    np_token = np.array(sent)
    str_token, str_token_count = np.unique(np_token, return_counts=True)
    for char, count in zip(str_token, str_token_count):
        if char not in token_map.keys():
            char = 'UNK'
        result[token_map[char]] = count
    return result

In [55]:
temp = np.vectorize(count_word, otypes=[object])(input_data)
x_f1 = np.array([[e for e in sl] for sl in temp.tolist()])

In [56]:
x_train=keras.preprocessing.sequence.pad_sequences(x_f1, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)

In [57]:
x_train.shape

(24511, 4957)

In [58]:
x_train, x_test =  train_test_split(x_train, test_size=0.2,random_state=1)

In [59]:
print("train size",x_train.shape)
print("test size",x_test.shape)

train size (19608, 4957)
test size (4903, 4957)


In [60]:
y_train = inputLabel
y_train = np.asarray(y_train).reshape(-1,1)

In [61]:
y_train, y_test =  train_test_split(y_train, test_size=0.2,random_state=1)

In [62]:
print("train size",len(y_train))
print("test size",len(y_test))

train size 19608
test size 4903


## Create Model

In [63]:
def create_model():    
    input1 = Input(shape=(x_train.shape[1],))
    x = Dense(1024, activation='relu')(input1)
    x = Dropout(0.5)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(1,activation='linear')(x)
    model = Model(inputs=input1, outputs=x)
    adam  = Adam(lr=0.0001)
    model.compile(optimizer=adam,  loss='mean_absolute_percentage_error')
    
    return model


In [64]:
model = create_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4957)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              5076992   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024

In [65]:
weight_path='./model_baseline_test1_withstop.h5'
callbacks_list = [
        ModelCheckpoint(
            weight_path,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
]

In [66]:
model.fit(x_train, y_train, batch_size=32 ,epochs=50, verbose=1, validation_split=0.2,callbacks=callbacks_list)

Train on 15686 samples, validate on 3922 samples
Epoch 1/50
15686/15686 [==============================] - 11s 699us/step - loss: 954.3705 - val_loss: 512.9504

Epoch 00001: val_loss improved from inf to 512.95036, saving model to ./model_baseline_test1_withstop.h5
Epoch 2/50
15686/15686 [==============================] - 9s 551us/step - loss: 773.2786 - val_loss: 377.1626

Epoch 00002: val_loss improved from 512.95036 to 377.16264, saving model to ./model_baseline_test1_withstop.h5
Epoch 3/50
15686/15686 [==============================] - 9s 549us/step - loss: 647.8586 - val_loss: 291.9656

Epoch 00003: val_loss improved from 377.16264 to 291.96558, saving model to ./model_baseline_test1_withstop.h5
Epoch 4/50
15686/15686 [==============================] - 9s 550us/step - loss: 531.7727 - val_loss: 238.9477

Epoch 00004: val_loss improved from 291.96558 to 238.94772, saving model to ./model_baseline_test1_withstop.h5
Epoch 5/50
15686/15686 [==============================] - 9s 543us/s

15686/15686 [==============================] - 9s 579us/step - loss: 38.5271 - val_loss: 71.9759

Epoch 00044: val_loss did not improve from 59.61376
Epoch 45/50
15686/15686 [==============================] - 9s 569us/step - loss: 38.0019 - val_loss: 72.2046

Epoch 00045: val_loss did not improve from 59.61376
Epoch 46/50
15686/15686 [==============================] - 8s 536us/step - loss: 37.5012 - val_loss: 71.7771

Epoch 00046: val_loss did not improve from 59.61376
Epoch 47/50
15686/15686 [==============================] - 8s 538us/step - loss: 36.9583 - val_loss: 73.3397

Epoch 00047: val_loss did not improve from 59.61376
Epoch 48/50
15686/15686 [==============================] - 8s 539us/step - loss: 36.7325 - val_loss: 72.8213

Epoch 00048: val_loss did not improve from 59.61376
Epoch 49/50
15686/15686 [==============================] - 9s 549us/step - loss: 36.6312 - val_loss: 73.2935

Epoch 00049: val_loss did not improve from 59.61376
Epoch 50/50
15686/15686 [===============

In [436]:
model.fit(x_train, y_train, batch_size=32 ,epochs=5, verbose=1, validation_split=0.2,callbacks=callbacks_list2)

Train on 15686 samples, validate on 3922 samples
Epoch 1/5
15686/15686 [==============================] - 9s 595us/step - loss: 36.5602 - val_loss: 72.7645
Epoch 2/5
  320/15686 [..............................] - ETA: 8s - loss: 33.7238

/home/burin/.env/src/keras/keras/callbacks.py:707: RuntimeWarning: Can save best model only with train_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


15686/15686 [==============================] - 9s 584us/step - loss: 36.2576 - val_loss: 73.2490
Epoch 3/5
15686/15686 [==============================] - 9s 588us/step - loss: 36.2094 - val_loss: 73.4807
Epoch 4/5
15686/15686 [==============================] - 9s 585us/step - loss: 35.4603 - val_loss: 74.0365
Epoch 5/5
15686/15686 [==============================] - 9s 555us/step - loss: 35.9419 - val_loss: 75.4226


In [ ]:
weight_model='./model_baseline_test2.h5'
model = create_model()
model.load_weights(weight_model)
model._make_predict_function()
model.summary()

## Evaluate Model

In [67]:
y_pred = model.predict(x_test)

In [68]:
y_pred2 = y_pred.clip(min=0)

In [69]:
def evaluate(preds, testY):
    diff = preds.flatten() - testY
    percentDiff = (diff / testY) * 100
    absPercentDiff = np.abs(percentDiff)
    mean = np.mean(absPercentDiff)
    std = np.std(absPercentDiff)
    print(mean,std)

In [70]:
evaluate(y_pred,y_test)

103.5437980900978 150.8995226451807


In [470]:
evaluate(y_pred,y_test)

97.73933719442356 147.63111233841977


In [430]:
evaluate(y_pred2,y_test)

88.36952848213208 128.7975869594508


In [471]:
for i in range(50):
    print(y_test2[i],y_pred2[i])
    print(y_test2[i],y_pred2[i]*MAXVIEW)

[104.] [0.04379225]
[104.] [153.09772]
[97.] [0.09940206]
[97.] [347.50958]
[94.] [0.05182158]
[94.] [181.16824]
[165.] [0.05189327]
[165.] [181.41887]
[724.] [0.11034767]
[724.] [385.77545]
[254.] [0.08795989]
[254.] [307.50778]
[1483.] [0.04845521]
[1483.] [169.39943]
[1490.] [0.1522605]
[1490.] [532.3027]
[360.] [0.11586411]
[360.] [405.06094]
[2805.] [0.11565819]
[2805.] [404.34103]
[3094.] [0.2553892]
[3094.] [892.8407]
[184.] [0.08681615]
[184.] [303.50928]
[231.] [0.16967946]
[231.] [593.1994]
[364.] [0.1624019]
[364.] [567.757]
[393.] [0.11865829]
[393.] [414.82938]
[479.] [0.10168976]
[479.] [355.50742]
[210.] [0.05912689]
[210.] [206.70763]
[423.] [0.0515598]
[423.] [180.25307]
[1278.] [0.07914282]
[1278.] [276.68332]
[2179.] [0.21793026]
[2179.] [761.88416]
[130.] [0.05533054]
[130.] [193.43558]
[73.] [0.06416036]
[73.] [224.30463]
[351.] [0.1459571]
[351.] [510.26602]
[127.] [0.05311427]
[127.] [185.68747]
[1788.] [0.1731593]
[1788.] [605.3649]
[195.] [0.06459923]
[195.] [2